## 2a. Evidence - Functional Correctness

Evidence collected in this section checks for functional correctness in the Gradient Climber Example

In [ ]:
{
    "tags": ["Reinforcement Learning"],
    "quality_attribute": "Functional Correctness",
    "description": "After initialization, the model successfully navigates the vehicle from a stationary position at the bottom of a hill, to the desired location (top of the hill in front of the vehicle) within the specified time bound.",
    "inputs": "Initial random start position",
    "output": "Log with 1 for each trial that completes within limits, and 0 for trials that do not.",
}

### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces. This import will also set up global constants related to folders and model to use.

In [ ]:
# Sets up context for the model being used, sets up constants related to folders and model data to be used.
from session import *

### Set up scnario test case

In [ ]:
from mlte.negotiation.artifact import NegotiationCard

card = NegotiationCard.load()
qa = 0
print(card.quality_scenarios[qa])

**A Specific test case generated from the scenario:**

**Data and Data Source:**	Vehicle state (position and velocity) from sensors (or approximated by simulation engine in development)

**Measurement and Condition:**	Variable  𝑝𝑜𝑠𝑖𝑡𝑖𝑜𝑛=0.6 for some  𝑡<250.

**Context:**	Normal Operation

### Helper Functions

In [ ]:
MEASURE_NAME = "functional correctness"
NUM_TRIALS = 100

In [ ]:
import numpy as np
import gymnasium as gym

In [ ]:
env = gym.make("MountainCar-v0", render_mode="rgb_array")
state, info = env.reset()

In [ ]:
# Discretize the state space (position, velocity)
position_bins = np.linspace(-1.2, 0.6, 20)
velocity_bins = np.linspace(-0.07, 0.07, 20)

# Q-table initialization
q_table = np.load(os.path.join(DATA_DIR, "mountain_car.npy"))


# Discretize the continuous state (position and velocity)
def discretize_state(state):
    position, velocity = state
    position_idx = (
        np.digitize(position, position_bins) - 1
    )  # Position bin index
    velocity_idx = (
        np.digitize(velocity, velocity_bins) - 1
    )  # Velocity bin index
    return position_idx, velocity_idx


# Epsilon-greedy action selection
def choose_action(state):
    position_idx, velocity_idx = discretize_state(state)
    return np.argmax(q_table[position_idx, velocity_idx])

## Reliability

Agent receives valid values from sensors during Normal Operation. Agent produces actions which improve the expected reward 99.9% of the time.

In [ ]:
def test_functional_correctness():
    done = False
    total_reward = 0
    test_results = []

    for i in range(NUM_TRIALS):
        state, info = env.reset()
        done = False
        actions = []

        while not done:
            # Random action selection
            action = choose_action(state)
            actions.append(action)

            # Take the action and get the next state, reward, done flag, and info
            next_state, reward, done, truncated, info = env.step(action)

            # Update the state for the next iteration
            state = next_state
        print(f"Completed trial {i}")

        if len(actions) < 250:
            test_results.append(1)
        else:
            test_results.append(0)

    return test_results

In [ ]:
from mlte.evidence.types.array import Array
from mlte.measurement.external_measurement import ExternalMeasurement

# Evaluate accuracy, identifier has to be the same one defined in the TestSuite.
position_compliance_measurement = ExternalMeasurement(
    MEASURE_NAME, Array, test_functional_correctness
)
evidence = position_compliance_measurement.evaluate()

# Inspect value
print(evidence)

# Save to artifact store
evidence.save(force=True, parents=True)